# Coding agent
A special case of Agent is a coding one, which is capable of running code. Options are Local and Azure (Dynamic Session). This notebook demonstrate the Azure one, which relies on [Dynamic Sessions](https://learn.microsoft.com/en-us/azure/container-apps/sessions?tabs=azure-cli)

**NOTE**: `AZURE_DYNAMIC_SESSIONS_ENDPOINT` env var must be set to the endpoint of the Azure Dynamic Sessions service, as describe in the documentation above. Additionally, this agent relies on Managed Identity authentication, so make sure:

1. You have Azure CLI installed
2. You are logged in with `az login` and selected the correct subscription with `az account set --subscription <subscription-id>`
3. You have the necessary permissions to access the Dynamic Session service (e.g. `az role assignment create --role "Azure ContainerApps Session Executor role" --assignee <your-identity> --scope <resource-group>`)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.workflow import Workflow
from vanilla_aiagents.team import Team
from vanilla_aiagents.llm import AzureOpenAILLM
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.team").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.coding_agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.llm").setLevel(logging.DEBUG)

In [ ]:
from vanilla_aiagents.azure_coding_agent import AzureCodingAgent
agent1 = AzureCodingAgent(id="agent1", description="Agent 1", llm=llm)

flow = Team(id="team", description="An agent capable of writing and executing code", members=[agent1], llm=llm, stop_callback=lambda conv: len(conv.messages) > 2)
workflow = Workflow(askable=flow)
workflow.conversation.variables["conversation_id"] = "test"

workflow.run("What is latest quote for Apple Inc. stock? Generate code to get the quote.")

workflow.conversation.messages